# Modelos Preditivos de Matemática Aplicada a Ciência de Dados

### Modelos Utilizados

## Classificação

- GaussianNB
- DecisionTreeClassifier
- KNeighborsClassifier

## Regressão

- LinearRegression

# Imports

In [29]:
## Modelos

### Modelos de classificação
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

### Modelos de Regressão
import numpy as np
from sklearn.linear_model import LinearRegression


## Processamento e normalização de dados
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler

## Testes e Avaliação
from sklearn.model_selection import train_test_split, GridSearchCV

## Métricas
from sklearn.metrics import accuracy_score, classification_report, r2_score,precision_score, recall_score, confusion_matrix, f1_score

from sklearn.base import is_classifier, is_regressor


## Processo de carregar e filtrar os dados do dataset

In [30]:
# Carregar o dataset em um dataframe
df = pd.read_csv('./datasets/predictive_maintenance.csv')

# Deletar colunas que não são uteis
df_processed = df.drop(['UDI', 'Product ID', 'Target'], axis=1)

# A coluna Failure Type é a coluna que vai ser preditada precisa ser convertida de texto para números

label_encoder = LabelEncoder()
df_processed['Failure Type'] = label_encoder.fit_transform(df_processed['Failure Type'])

# A coluna 'Type' é uma feature. Vamos usar a codificação one-hot para convertê-la
# Isso impede que o modelo pense que 'L' < 'M' < 'H'.
df_processed = pd.get_dummies(df_processed, columns=['Type'], drop_first=True)

# Separando as colunas preditada e o resto
X = df_processed.drop('Failure Type', axis=1)
y = df_processed['Failure Type']

print("\nProcessed Features (X) head:")
print(X.head())


Processed Features (X) head:
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0                298.1                    308.6                    1551   
1                298.2                    308.7                    1408   
2                298.1                    308.5                    1498   
3                298.2                    308.6                    1433   
4                298.2                    308.7                    1408   

   Torque [Nm]  Tool wear [min]  Type_L  Type_M  
0         42.8                0   False    True  
1         46.3                3    True   False  
2         49.4                5    True   False  
3         39.5                7    True   False  
4         40.0                9    True   False  


In [31]:
display(df_processed)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,1,False,True
1,298.2,308.7,1408,46.3,3,1,True,False
2,298.1,308.5,1498,49.4,5,1,True,False
3,298.2,308.6,1433,39.5,7,1,True,False
4,298.2,308.7,1408,40.0,9,1,True,False
...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,1,False,True
9996,298.9,308.4,1632,31.8,17,1,False,False
9997,299.0,308.6,1645,33.4,22,1,False,True
9998,299.0,308.7,1408,48.5,25,1,False,False


In [32]:
df_processed['Failure Type'].unique()

array([1, 3, 5, 2, 4, 0])

In [33]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"\nQuantidade de linhas de Treinamento: {X_train.shape}")
print(f"\nQuantidade de linhas de Teste: {X_test.shape}")


Quantidade de linhas de Treinamento: (8000, 7)

Quantidade de linhas de Teste: (2000, 7)


In [34]:

# Colunas que são numericas e precisam de normalização

numerical_features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

#Inicializar o scaler
scaler = StandardScaler()

# Colocar o scaler para aprender os parâmetros de normalização (média e desvio padrão) a partir dos dados de treino
scaler.fit(X_train[numerical_features])

# Transoformar os dados de treino e teste
X_train[numerical_features] = scaler.transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

print("\nX_train normalizado:")
print(X_train.head())


X_train normalizado:
      Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
9254            -0.854066                -0.609589                0.427634   
1561            -0.904014                -1.080528               -0.834945   
1670            -0.904014                -1.484190               -0.059677   
6087             0.444571                 0.534121                0.333495   
6669             0.694309                 0.332290                0.178441   

      Torque [Nm]  Tool wear [min]  Type_L  Type_M  
9254    -0.892696         1.375035    True   False  
1561     1.382187         0.457620    True   False  
1670    -0.892696         1.359218    True   False  
6087    -0.702288        -1.598655   False    True  
6669    -0.612094         1.580663    True   False  


## Classificação do modelo


In [35]:
# --- Determinar automaticamente o tipo de problema ---
# Número de valores distintos na variável alvo
unique_values = y.nunique()
print(f"\nValores únicos na variável alvo: {unique_values}")
print(f"Exemplos de valores: {y.unique()[:10]}")

# Verifica o tipo de problema
if y.dtype == 'object' or unique_values < 20:
    problema = "classificacao"
else:
    problema = "regressao"

print(f"\nTipo de problema detectado: {problema.upper()}")



Valores únicos na variável alvo: 6
Exemplos de valores: [1 3 5 2 4 0]

Tipo de problema detectado: CLASSIFICACAO


# Modelos


## Naive Bayes

In [36]:
naive_bayes = GaussianNB()

param_grid_Gaussian_NB = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

gs_nb = GridSearchCV(estimator=naive_bayes, param_grid=param_grid_Gaussian_NB, scoring='accuracy', cv=5, n_jobs=-1)
gs_nb.fit(X_train, y_train)

naive_bayes = GaussianNB()

naive_bayes.fit(X_train, y_train)

# Usa o melhor modelo encontrado pelo GridSearchCV para fazer previsões
best_nb = gs_nb.best_estimator_
y_pred_nb = best_nb.predict(X_test)

## Decision Tree

In [37]:
decision_tree = DecisionTreeClassifier(random_state=42)

param_grid_Decision_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}


gs_DT = GridSearchCV(estimator=decision_tree, param_grid=param_grid_Decision_tree, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)

# Treina o GridSearchCV para encontrar os melhores parâmetros
gs_DT.fit(X_train, y_train)

# Mostra os melhores parâmetros encontrados
print("Melhores parâmetros para Decision Tree: ", gs_DT.best_params_)

# Usa o melhor modelo encontrado pelo GridSearchCV para fazer previsões
best_tree = gs_DT.best_estimator_
y_pred_dt = best_tree.predict(X_test)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Melhores parâmetros para Decision Tree:  {'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 2}


## KNN 

In [38]:
knn = KNeighborsClassifier()

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

gs_KNN = GridSearchCV(estimator=knn, param_grid=param_grid_knn, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)
gs_KNN.fit(X_train, y_train)


# Mostra os melhores parâmetros encontrados
print("Melhores parâmetros para KNN: ", gs_KNN.best_params_)

# Usa o melhor modelo encontrado pelo GridSearchCV para fazer previsões
best_knn = gs_KNN.best_estimator_
y_pred_knn = best_knn.predict(X_test)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Melhores parâmetros para KNN:  {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


## Regressão Linear

In [39]:
linear = LinearRegression()

param_grid_Linear_Regression = {
    'fit_intercept': [True, False],
    'copy_X': [True, False]
}

gs_LR = GridSearchCV(estimator=linear, param_grid=param_grid_Linear_Regression, scoring='r2', cv=5, n_jobs=-1, verbose=1)
gs_LR.fit(X_train, y_train)

print("Melhores parâmetros para Regressão Linear: ", gs_LR.best_params_)

best_lr = gs_LR.best_estimator_
y_pred_lr = best_lr.predict(X_test)

y_pred_lr = np.round(y_pred_lr).astype(int)

y_pred_lr = np.clip(y_pred_lr, y.min(), y.max())

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores parâmetros para Regressão Linear:  {'copy_X': True, 'fit_intercept': True}


## RandomForestClassifier

In [40]:
randomForest = RandomForestClassifier()

param_grid_Random_forest = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced']
}


gs_RF = GridSearchCV(estimator=linear, param_grid=param_grid_Linear_Regression, scoring='r2', cv=5, n_jobs=-1, verbose=1 )
gs_RF.fit(X_train, y_train)


print("Melhores parâmetros para Regressão Linear: ", gs_RF.best_params_)

best_rf = gs_RF.best_estimator_
y_pred_rf = best_rf.predict(X_test)

y_pred_rf = np.round(y_pred_rf).astype(int)

y_pred_rf = np.clip(y_pred_rf, y.min(), y.max())

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores parâmetros para Regressão Linear:  {'copy_X': True, 'fit_intercept': True}


## Modelos 

In [ ]:
if problema == "classificacao":
    models = {
        "GaussianNB": (
            GaussianNB(),
            {"var_smoothing": np.logspace(0, -9, num=100)}
        ),
        "DecisionTree": (
            DecisionTreeClassifier(random_state=42),
            {
                "criterion": ["gini", "entropy"],
                "max_depth": [None, 10, 20, 30],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
                "max_features": ["sqrt", "log2", None]
            }
        ),
        "KNN": (
            KNeighborsClassifier(),
            {
                "n_neighbors": [3, 5, 7, 9, 11],
                "weights": ["uniform", "distance"],
                "metric": ["euclidean", "manhattan"]
            }
        ),
        "RandomForest": (
            RandomForestClassifier(),
            {
                'n_estimators': [100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'max_features': ['sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
            }
        )
    }

else:  # problema == "regressao"
    models = {
        "LinearRegression": (
            LinearRegression(),
            {
                'fit_intercept': [True, False],
                'copy_X': [True, False]
            }
        )
    }


# Dicionários para armazenar resultados
best_models = {}
metrics = {}

# Loop para treinar e avaliar cada modelo
for name, (model, params) in models.items():
    print(f"\nTreinando {name}...")

    # Detecta automaticamente o tipo do modelo
    modelo_tipo = "regressor" if is_regressor(model) else "classificador"

    # Define a métrica de pontuação correta
    scoring_metric = 'r2' if modelo_tipo == "regressor" else 'accuracy'

    gs = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring=scoring_metric,
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)

    print(f"Melhores parâmetros para {name}: {gs.best_params_}")

    best_model = gs.best_estimator_
    best_models[name] = best_model

    # Previsões no conjunto de teste
    y_pred = best_model.predict(X_test)

    # prepara dicionário com todas as chaves (garante consistência entre modelos)
    model_metrics = {
        "Accuracy": np.nan,
        "Precision": np.nan,
        "Recall": np.nan,
        "F1-Score": np.nan,
        "Confusion Matrix": None,
        "R²": np.nan,
    }

    # Se for regressão, converte previsões contínuas para rótulos e calcula métricas de classificação + R²
    if modelo_tipo == "regressor":
        # arredonda para o inteiro mais próximo e limita ao intervalo das classes existentes
        y_pred_labels = np.rint(y_pred).astype(int)
        y_pred_labels = np.clip(y_pred_labels, int(y_test.min()), int(y_test.max()))

        # métricas de regressão
        r2 = r2_score(y_test, y_pred)
        model_metrics["R²"] = r2

        # métricas de classificação usando as previsões convertidas
        acc = accuracy_score(y_test, y_pred_labels)
        prec = precision_score(y_test, y_pred_labels, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred_labels, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred_labels, average='weighted', zero_division=0)
        cm = confusion_matrix(y_test, y_pred_labels, labels=np.unique(y_test))

        model_metrics["Accuracy"] = acc
        model_metrics["Precision"] = prec
        model_metrics["Recall"] = rec
        model_metrics["F1-Score"] = f1
        model_metrics["Confusion Matrix"] = cm

    else:
        # Classificação padrão
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

        model_metrics["Accuracy"] = acc
        model_metrics["Precision"] = prec
        model_metrics["Recall"] = rec
        model_metrics["F1-Score"] = f1
        model_metrics["Confusion Matrix"] = cm

    # salva métricas padronizadas
    metrics[name] = model_metrics

# Exibe resumo geral das métricas
summary_df = pd.DataFrame(metrics).T

# Remove a coluna da matriz de confusão apenas se ela existir
if "Confusion Matrix" in summary_df.columns:
    summary = summary_df.drop(columns="Confusion Matrix")
else:
    summary = summary_df



Treinando LinearRegression...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores parâmetros para LinearRegression: {'copy_X': True, 'fit_intercept': True}


# Métricas de Avaliação

In [42]:

print("\n\nResumo Final das Métricas:\n")
print(summary)



Resumo Final das Métricas:

                 Accuracy Precision  Recall  F1-Score        R²
LinearRegression   0.9665  0.938373  0.9665  0.952229  0.052633


##